In [2]:
import torch

def read_pt(filename):
    with open(filename, "rb") as f:
        return torch.load(f)
print(torch.__path__)
file_path = "/data/tianyuhu/LayoutGeneration/LayoutPrompter/dataset/rico/raw/test.pt"

data = read_pt(file_path)

['/data/tianyuhu/anaconda3/envs/pptlayout/lib/python3.11/site-packages/torch']


/tmp/ipykernel_835129/2244764574.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f)


In [3]:
print(data)

[{'name': '15468.json', 'bboxes': tensor([[0.3806, 0.3473, 0.2382, 0.0535],
        [0.2576, 0.1887, 0.0597, 0.0535],
        [0.3729, 0.1102, 0.2583, 0.0559],
        [0.0000, 0.8687, 0.2139, 0.0656],
        [0.2139, 0.8945, 0.5722, 0.0137],
        [0.7861, 0.8687, 0.2139, 0.0656],
        [0.1000, 0.0410, 0.8000, 0.5348],
        [0.0000, 0.5758, 1.0000, 0.0504],
        [0.0000, 0.6262, 1.0000, 0.0398]]), 'labels': tensor([ 2,  2,  2,  5, 16,  5,  2,  1,  1]), 'canvas_size': [1440.0, 2560.0], 'filtered': False}, {'name': '2216.json', 'bboxes': tensor([[0.0000, 0.0351, 1.0000, 0.1559],
        [0.0243, 0.2805, 0.9514, 0.0702],
        [0.0243, 0.3508, 0.9514, 0.0702],
        [0.0243, 0.4636, 0.9514, 0.0702],
        [0.0243, 0.5589, 0.9514, 0.0702]]), 'labels': tensor([2, 8, 8, 5, 5]), 'canvas_size': [1440.0, 2392.0], 'filtered': False}, {'name': '15965.json', 'bboxes': tensor([[0.0000, 0.0328, 1.0000, 0.9016],
        [0.2535, 0.0465, 0.4931, 0.0324],
        [0.4590, 0.0789, 0.0